O que fazer:
- em IMDbBasics, adicionar os títulos (tconst) dos episódios (IMDbEpisode) (PS: averiguar runtimeMinutes)
- em NetflixIMDb, mover os atributos de episódio em IMDbEpisode

Power BI:
- Adicionar atributo de weekday
- Adicionar atributo de ano (para saber os 3 títulos em cada ano)

# Importa as bibliotecas da aplicação

In [1]:
!pip install pyspark

     |████████████████████████████████| 281.4 MB 34 kB/s 
     |████████████████████████████████| 198 kB 58.5 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=88b26faebd6e0579602b306f2a8159458de4e9757824cb700833cf10b161129e
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [2]:
import sys

from IPython.display import display
from pyspark.conf import SparkConf
from pyspark.sql import SparkSession
from pyspark.context import SparkContext

from IPython.display import HTML, display
import tabulate

import pandas as pd
import numpy as np
import re

from google.colab import files

pd.options.display.max_colwidth = 100
pd.set_option('display.max_rows', None)

# Configura o ambiente de execução do Apache Spark

In [3]:
spark = SparkSession \
        .builder \
        .master("local[2]") \
        .appName("Netflix Analysis") \
        .config("spark.executor.cores", "2") \
        .config("spark.executor.memory", "2g") \
        .config("spark.cores.max", "4") \
        .getOrCreate()

# Realiza a leituras dos dados

In [4]:
!wget https://datasets.imdbws.com/title.akas.tsv.gz

--2022-03-19 20:29:16--  https://datasets.imdbws.com/title.akas.tsv.gz
Resolving datasets.imdbws.com (datasets.imdbws.com)... 13.225.142.7, 13.225.142.81, 13.225.142.61, ...
Connecting to datasets.imdbws.com (datasets.imdbws.com)|13.225.142.7|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 266407475 (254M) [binary/octet-stream]
Saving to: ‘title.akas.tsv.gz’

title.akas.tsv.gz   100%[===================>] 254.07M  96.8MB/s    in 2.6s    

2022-03-19 20:29:19 (96.8 MB/s) - ‘title.akas.tsv.gz’ saved [266407475/266407475]



In [5]:
!wget https://datasets.imdbws.com/title.ratings.tsv.gz

--2022-03-19 20:29:19--  https://datasets.imdbws.com/title.ratings.tsv.gz
Resolving datasets.imdbws.com (datasets.imdbws.com)... 13.225.142.7, 13.225.142.81, 13.225.142.61, ...
Connecting to datasets.imdbws.com (datasets.imdbws.com)|13.225.142.7|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6116207 (5.8M) [binary/octet-stream]
Saving to: ‘title.ratings.tsv.gz’

title.ratings.tsv.g 100%[===================>]   5.83M  --.-KB/s    in 0.1s    

2022-03-19 20:29:19 (56.1 MB/s) - ‘title.ratings.tsv.gz’ saved [6116207/6116207]



In [6]:
!wget https://datasets.imdbws.com/title.basics.tsv.gz

--2022-03-19 20:29:19--  https://datasets.imdbws.com/title.basics.tsv.gz
Resolving datasets.imdbws.com (datasets.imdbws.com)... 13.225.142.7, 13.225.142.81, 13.225.142.61, ...
Connecting to datasets.imdbws.com (datasets.imdbws.com)|13.225.142.7|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 153660815 (147M) [binary/octet-stream]
Saving to: ‘title.basics.tsv.gz’

title.basics.tsv.gz 100%[===================>] 146.54M  77.7MB/s    in 1.9s    

2022-03-19 20:29:21 (77.7 MB/s) - ‘title.basics.tsv.gz’ saved [153660815/153660815]



In [7]:
!wget https://datasets.imdbws.com/title.episode.tsv.gz

--2022-03-19 20:29:21--  https://datasets.imdbws.com/title.episode.tsv.gz
Resolving datasets.imdbws.com (datasets.imdbws.com)... 13.225.142.7, 13.225.142.81, 13.225.142.61, ...
Connecting to datasets.imdbws.com (datasets.imdbws.com)|13.225.142.7|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 35661361 (34M) [binary/octet-stream]
Saving to: ‘title.episode.tsv.gz’

title.episode.tsv.g 100%[===================>]  34.01M   138MB/s    in 0.2s    

2022-03-19 20:29:22 (138 MB/s) - ‘title.episode.tsv.gz’ saved [35661361/35661361]



In [8]:
!gzip -d title.akas.tsv.gz

In [9]:
!gzip -d title.ratings.tsv.gz

In [10]:
!gzip -d title.basics.tsv.gz

In [11]:
!gzip -d title.episode.tsv.gz

In [12]:
!ls

sample_data	title.basics.tsv   title.ratings.tsv
title.akas.tsv	title.episode.tsv


In [13]:
# IMDb Title Akas DataFrame
akas_df = spark.read.options(delimiter='\t').option("header", True).csv("/content/title.akas.tsv")

In [14]:
# IMDb Title Ratings DataFrame
ratings_df = spark.read.options(delimiter='\t').option("header", True).csv("/content/title.ratings.tsv")

In [15]:
# IMDb Title Basics DataFrame
basics_df = spark.read.options(delimiter='\t').option("header", True).csv("/content/title.basics.tsv")

In [16]:
# IMDb Title Episode DataFrame
episode_df = spark.read.options(delimiter='\t').option("header", True).csv("/content/title.episode.tsv")

In [17]:
# Netflix Streaming History DataFrame
netflix_df = pd.read_csv('https://raw.githubusercontent.com/vichShir/netflix-analysis/master/data/Netflix/NetflixViewingHistory2022.csv')

# Confere a estrutura dos dados

In [ ]:
print(type(akas_df))
print(type(ratings_df))
print(type(basics_df))
print(type(episode_df))

<class 'pyspark.sql.dataframe.DataFrame'>
<class 'pyspark.sql.dataframe.DataFrame'>
<class 'pyspark.sql.dataframe.DataFrame'>
<class 'pyspark.sql.dataframe.DataFrame'>


In [ ]:
akas_df.printSchema()

root
 |-- titleId: string (nullable = true)
 |-- ordering: string (nullable = true)
 |-- title: string (nullable = true)
 |-- region: string (nullable = true)
 |-- language: string (nullable = true)
 |-- types: string (nullable = true)
 |-- attributes: string (nullable = true)
 |-- isOriginalTitle: string (nullable = true)



In [ ]:
print(f'DataFrame Title Akas tem {akas_df.count()} linhas.')

DataFrame Title Akas tem 31268700 linhas.


In [ ]:
ratings_df.printSchema()

root
 |-- tconst: string (nullable = true)
 |-- averageRating: string (nullable = true)
 |-- numVotes: string (nullable = true)



In [ ]:
print(f'DataFrame Title Ratings tem {ratings_df.count()} linhas.')

DataFrame Title Ratings tem 1221670 linhas.


In [ ]:
basics_df.printSchema()

root
 |-- tconst: string (nullable = true)
 |-- titleType: string (nullable = true)
 |-- primaryTitle: string (nullable = true)
 |-- originalTitle: string (nullable = true)
 |-- isAdult: string (nullable = true)
 |-- startYear: string (nullable = true)
 |-- endYear: string (nullable = true)
 |-- runtimeMinutes: string (nullable = true)
 |-- genres: string (nullable = true)



In [ ]:
print(f'DataFrame Title Basics tem {basics_df.count()} linhas.')

DataFrame Title Basics tem 8750730 linhas.


In [ ]:
episode_df.printSchema()

root
 |-- tconst: string (nullable = true)
 |-- parentTconst: string (nullable = true)
 |-- seasonNumber: string (nullable = true)
 |-- episodeNumber: string (nullable = true)



In [ ]:
print(f'DataFrame Title Episode tem {episode_df.count()} linhas.')

DataFrame Title Episode tem 6552731 linhas.


In [ ]:
netflix_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 450 entries, 0 to 449
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Title   450 non-null    object
 1   Date    450 non-null    object
dtypes: object(2)
memory usage: 7.2+ KB


In [ ]:
sum(netflix_df.duplicated())

0

# Pré-visualização dos dados

In [ ]:
# IMDb Title Akas DataFrame
display(akas_df.limit(5).toPandas())

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,1,Карменсіта,UA,\N,imdbDisplay,\N,0
1,tt0000001,2,Carmencita,DE,\N,\N,literal title,0
2,tt0000001,3,Carmencita - spanyol tánc,HU,\N,imdbDisplay,\N,0
3,tt0000001,4,Καρμενσίτα,GR,\N,imdbDisplay,\N,0
4,tt0000001,5,Карменсита,RU,\N,imdbDisplay,\N,0


In [ ]:
# IMDb Title Ratings DataFrame
display(ratings_df.limit(5).toPandas())

,tconst,averageRating,numVotes
0,tt0000001,5.7,1864
1,tt0000002,6.0,244
2,tt0000003,6.5,1632
3,tt0000004,5.8,158
4,tt0000005,6.2,2458


In [ ]:
# IMDb Title Basics DataFrame
display(basics_df.limit(5).toPandas())

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


In [ ]:
# IMDb Title Episode DataFrame
display(episode_df.limit(5).toPandas())

,tconst,parentTconst,seasonNumber,episodeNumber
0,tt0020666,tt15180956,1,2
1,tt0020829,tt15180956,1,1
2,tt0021166,tt15180956,1,3
3,tt0021612,tt15180956,2,2
4,tt0021655,tt15180956,2,5


In [ ]:
# Netflix Streaming History DataFrame
netflix_df.head()

,Title,Date
0,Segunda Guerra em Cores: Caminho para a Vitória: Temporada 1: Ocupação da Itália,14/02/2022
1,Segunda Guerra em Cores: Caminho para a Vitória: Temporada 1: Batalha de Kursk,08/02/2022
2,Segunda Guerra em Cores: Caminho para a Vitória: Temporada 1: Ocupação do Norte da África,31/01/2022
3,Segunda Guerra em Cores: Caminho para a Vitória: Temporada 1: Batalha do Atlântico,30/12/2021
4,Segunda Guerra em Cores: Caminho para a Vitória: Temporada 1: Dunquerque,27/12/2021


# Separar o título e o nome do episódio

In [18]:
netflix_df['Title']

0               Segunda Guerra em Cores: Caminho para a Vitória: Temporada 1: Ocupação da Itália
1                 Segunda Guerra em Cores: Caminho para a Vitória: Temporada 1: Batalha de Kursk
2      Segunda Guerra em Cores: Caminho para a Vitória: Temporada 1: Ocupação do Norte da África
3             Segunda Guerra em Cores: Caminho para a Vitória: Temporada 1: Batalha do Atlântico
4                       Segunda Guerra em Cores: Caminho para a Vitória: Temporada 1: Dunquerque
5                            Grandes Momentos da Segunda Guerra em Cores: Temporada 1: Hiroshima
6             Grandes Momentos da Segunda Guerra em Cores: Temporada 1: Libertação de Buchenwald
7                Grandes Momentos da Segunda Guerra em Cores: Temporada 1: Bombardeio de Dresden
8                     Grandes Momentos da Segunda Guerra em Cores: Temporada 1: Batalha do Bulge
9                                Grandes Momentos da Segunda Guerra em Cores: Temporada 1: Dia D
10                Grandes Mome

In [19]:
# Remove duplicated row streaming error
netflix_df = netflix_df[netflix_df['Title'] != 'Mars: Season 1_hook_04_16x9']

In [20]:
def split_title(title):
  """
    Return the main title and episode name by title
  """
  main_title = title
  episode_name = ''
  texts = re.split('Temporada |Minissérie', title)

  # title have the separator key
  if len(texts) > 1:
    # get the main title
    main_title = texts[0]
    # remove white spaces
    main_title = main_title.strip()
    # remove the ':' character
    main_title = main_title[:-1]

    # get the episode name
    episode_name = texts[1]

  return (main_title, episode_name)

In [21]:
# Get episode name from Title
netflix_df['Episode'] = netflix_df['Title'].map(lambda x: split_title(x)[1])

In [22]:
# Update Title to the main title
netflix_df['Title'] = netflix_df['Title'].map(lambda x: split_title(x)[0])

In [23]:
netflix_df.head()

,Title,Date,Episode
0,Segunda Guerra em Cores: Caminho para a Vitória,14/02/2022,1: Ocupação da Itália
1,Segunda Guerra em Cores: Caminho para a Vitória,08/02/2022,1: Batalha de Kursk
2,Segunda Guerra em Cores: Caminho para a Vitória,31/01/2022,1: Ocupação do Norte da África
3,Segunda Guerra em Cores: Caminho para a Vitória,30/12/2021,1: Batalha do Atlântico
4,Segunda Guerra em Cores: Caminho para a Vitória,27/12/2021,1: Dunquerque


# Associar um ID do IMDb

In [24]:
# Get unique titles
titles = netflix_df['Title'].unique()
titles = np.sort(titles)
titles

array(['A Garota que conquistou o tempo', 'A Viagem de Chihiro',
       'A Voz do Silêncio – Koe No Katachi', 'Ano Hana',
       'Atlantis - O reino perdido', 'Bob Esponja',
       "Bob Esponja - Um Herói Fora D'Água", 'Challenger: Voo Final',
       'Coraline e o mundo secreto', 'Crianças Lobo', 'Divertida Mente',
       'Escola de rock', 'Explicando... O Coronavírus',
       'Exterminadores do Além Contra a Loira do Banheiro',
       'Grandes Momentos da Segunda Guerra em Cores',
       'História: Direto ao Assunto', 'Kubo e as Cordas Mágicas',
       'Little Witch Academia',
       'Little Witch Academia: The Enchanted Parade',
       'Luzes no Céu: Fireworks', 'Marte', 'Meu Amigo Totoro',
       'Mission Control: The Unsung Heroes of Apollo',
       'NOVA: Black Hole Apocalypse', 'NOVA: Viagem a Saturno',
       'O Universo', 'Olhos de Gato', 'One-Punch Man', 'ParaNorman',
       'Pets - A Vida Secreta dos Bichos', 'Sangatsu no Lion',
       'Segunda Guerra em Cores: Caminho para a

In [25]:
print(f'São ao todo {len(titles)} títulos.')

São ao todo 44 títulos.


In [26]:
# Registering a table
akas_df.registerTempTable("akas_df")
ratings_df.registerTempTable("ratings_df")
basics_df.registerTempTable("basics_df")
episode_df.registerTempTable("episode_df")

/usr/local/lib/python3.7/dist-packages/pyspark/sql/dataframe.py:140: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  FutureWarning


In [27]:
original_titles = np.array(['Toki o kakeru shôjo', 'Sen to Chihiro no kamikakushi', 'Koe no katachi', 'Anohana: The Flower We Saw That Day',
                            'Atlantis: The Lost Empire', 'SpongeBob SquarePants', 'The SpongeBob Movie: Sponge Out of Water', 'Challenger: The Final Flight',
                            'Coraline', 'Wolf Children', 'Inside Out', 'School of Rock',
                            'Coronavirus, Explained', 'Exterminadores do Além Contra a Loira do Banheiro', 'Greatest Events of WWII in Colour', 'History 101',
                            'Kubo and the Two Strings', 'Little Witch Academia', 'Little Witch Academia: The Enchanted Parade', 'Luzes no Céu: Fireworks',
                            'The Martian', 'Tonari no Totoro', 'Mission Control: The Unsung Heroes of Apollo', 'Nova', 'Nova',
                            'The Universe', 'A Whisker Away', 'One Punch Man: Wanpanman', 'ParaNorman',
                            'The Secret Life of Pets', 'Sangatsu no Lion', 'WWII in Color: Road to Victory', 'Sherlock',
                            'Flavors of Youth', 'Space Force', 'Karakai Jouzu no Takagi-san', 'The Search for Life in Space',
                            'The Walking Dead', 'Toradora!', 'Tron: Legacy', 'Full House',
                            'The Fresh Prince of Bel-Air', 'Children Who Chase Lost Voices', 'Kimi no na wa.'])
original_titles

array(['Toki o kakeru shôjo', 'Sen to Chihiro no kamikakushi',
       'Koe no katachi', 'Anohana: The Flower We Saw That Day',
       'Atlantis: The Lost Empire', 'SpongeBob SquarePants',
       'The SpongeBob Movie: Sponge Out of Water',
       'Challenger: The Final Flight', 'Coraline', 'Wolf Children',
       'Inside Out', 'School of Rock', 'Coronavirus, Explained',
       'Exterminadores do Além Contra a Loira do Banheiro',
       'Greatest Events of WWII in Colour', 'History 101',
       'Kubo and the Two Strings', 'Little Witch Academia',
       'Little Witch Academia: The Enchanted Parade',
       'Luzes no Céu: Fireworks', 'The Martian', 'Tonari no Totoro',
       'Mission Control: The Unsung Heroes of Apollo', 'Nova', 'Nova',
       'The Universe', 'A Whisker Away', 'One Punch Man: Wanpanman',
       'ParaNorman', 'The Secret Life of Pets', 'Sangatsu no Lion',
       'WWII in Color: Road to Victory', 'Sherlock', 'Flavors of Youth',
       'Space Force', 'Karakai Jouzu no Takag

In [28]:
mapped_titles = dict(zip(titles, original_titles))
mapped_titles

{'A Garota que conquistou o tempo': 'Toki o kakeru shôjo',
 'A Viagem de Chihiro': 'Sen to Chihiro no kamikakushi',
 'A Voz do Silêncio – Koe No Katachi': 'Koe no katachi',
 'Ano Hana': 'Anohana: The Flower We Saw That Day',
 'Atlantis - O reino perdido': 'Atlantis: The Lost Empire',
 'Bob Esponja': 'SpongeBob SquarePants',
 "Bob Esponja - Um Herói Fora D'Água": 'The SpongeBob Movie: Sponge Out of Water',
 'Challenger: Voo Final': 'Challenger: The Final Flight',
 'Coraline e o mundo secreto': 'Coraline',
 'Crianças Lobo': 'Wolf Children',
 'Divertida Mente': 'Inside Out',
 'Escola de rock': 'School of Rock',
 'Explicando... O Coronavírus': 'Coronavirus, Explained',
 'Exterminadores do Além Contra a Loira do Banheiro': 'Exterminadores do Além Contra a Loira do Banheiro',
 'Grandes Momentos da Segunda Guerra em Cores': 'Greatest Events of WWII in Colour',
 'História: Direto ao Assunto': 'History 101',
 'Kubo e as Cordas Mágicas': 'Kubo and the Two Strings',
 'Little Witch Academia': 'Lit

In [29]:
netflix_df['Title'].replace(mapped_titles, inplace=True)
netflix_df.sample(5)

,Title,Date,Episode
412,Little Witch Academia,29/09/2019,1: Começando do zero
192,Full House,14/01/2020,3: Onde está o passarinho que estava aqui?
415,Anohana: The Flower We Saw That Day,21/09/2019,1: Episódio 10
262,SpongeBob SquarePants,16/12/2019,6: Casa Chique / Estrada do Carsiri
23,The Walking Dead,08/05/2021,8: Um cara aí


In [30]:
from IPython.display import clear_output

def get_id_from_imdb(title_name):
  titles = spark.sql(f"SELECT titleId FROM akas_df WHERE title = '{title_name}' AND types = 'imdbDisplay'").toPandas()
  titles = titles.drop_duplicates(subset=['titleId'])
  titles = titles.reset_index(drop=True)

  """
  ids_size = len(titles.index)

  if ids_size > 1:
    clear_output(wait=True)
    print(f'Escolha um ID (index) para <<{title_name}>>:')
    display(titles)
    input_id = int(input('>> '))
    id = titles['titleId'][titles.index[input_id]]
  elif ids_size == 1:
    id = titles['titleId'][titles.index[0]]
  else:
    id = None
  """

  if len(titles) > 0:
    id = titles['titleId'][titles.index[0]]
  else:
    id = None

  title_id = id if id != None else np.nan
  return title_id

In [ ]:
title_ids = [get_id_from_imdb(x) for x in original_titles]
title_ids

['tt0808506',
 'tt0245429',
 'tt5323662',
 'tt1913273',
 'tt0230011',
 'tt0206512',
 'tt2279373',
 'tt12930534',
 'tt0327597',
 'tt2140203',
 'tt0073174',
 'tt0332379',
 'tt12189310',
 'tt8753660',
 'tt9103932',
 'tt11958648',
 'tt4302938',
 'tt6352180',
 'tt4831682',
 'tt6317962',
 'tt2910906',
 'tt0096283',
 'tt5959952',
 'tt0206501',
 'tt0206501',
 'tt1051155',
 'tt11958344',
 'tt4508902',
 'tt1623288',
 'tt2709768',
 'tt6074794',
 'tt16477402',
 'tt0337593',
 'tt8176578',
 'tt11309924',
 'tt7808344',
 'tt7521400',
 'tt0028478',
 'tt1279024',
 'tt1104001',
 'tt0092359',
 'tt0098800',
 'tt1839494',
 'tt5311514']

In [31]:
# Shortcut for find IDs
title_ids = ['tt0808506',
 'tt0245429',
 'tt5323662',
 'tt1913273',
 'tt0230011',
 'tt0206512',
 'tt2279373',
 'tt12930534',
 'tt0327597',
 'tt2140203',
 'tt0073174',
 'tt0332379',
 'tt12189310',
 'tt8753660',
 'tt9103932',
 'tt11958648',
 'tt4302938',
 'tt6352180',
 'tt4831682',
 'tt6317962',
 'tt2910906',
 'tt0096283',
 'tt5959952',
 'tt0206501',
 'tt0206501',
 'tt1051155',
 'tt11958344',
 'tt4508902',
 'tt1623288',
 'tt2709768',
 'tt6074794',
 'tt16477402',
 'tt0337593',
 'tt8176578',
 'tt11309924',
 'tt7808344',
 'tt7521400',
 'tt0028478',
 'tt1279024',
 'tt1104001',
 'tt0092359',
 'tt0098800',
 'tt1839494',
 'tt5311514']

## Associar IDs nos dados do Netflix

In [34]:
def categorise(row):
  for i in range(len(original_titles)):
    if(row['Title'] == original_titles[i]):
      return title_ids[i]
  return np.nan

In [35]:
netflix_df['titleId'] = netflix_df.apply(lambda row: categorise(row), axis=1)
netflix_df.head()

,Title,Date,Episode,titleId
0,WWII in Color: Road to Victory,14/02/2022,1: Ocupação da Itália,tt16477402
1,WWII in Color: Road to Victory,08/02/2022,1: Batalha de Kursk,tt16477402
2,WWII in Color: Road to Victory,31/01/2022,1: Ocupação do Norte da África,tt16477402
3,WWII in Color: Road to Victory,30/12/2021,1: Batalha do Atlântico,tt16477402
4,WWII in Color: Road to Victory,27/12/2021,1: Dunquerque,tt16477402


## Tratar o nome dos episódios

In [36]:
# Extract season number
netflix_df['seasonNumber'] = netflix_df['Episode'].str.extract(r'^[^\d]*(\d+)')
netflix_df.head()

,Title,Date,Episode,titleId,seasonNumber
0,WWII in Color: Road to Victory,14/02/2022,1: Ocupação da Itália,tt16477402,1
1,WWII in Color: Road to Victory,08/02/2022,1: Batalha de Kursk,tt16477402,1
2,WWII in Color: Road to Victory,31/01/2022,1: Ocupação do Norte da África,tt16477402,1
3,WWII in Color: Road to Victory,30/12/2021,1: Batalha do Atlântico,tt16477402,1
4,WWII in Color: Road to Victory,27/12/2021,1: Dunquerque,tt16477402,1


In [37]:
# Remove special characters at the beginning
netflix_df['Episode'] = netflix_df['Episode'].map(lambda x: x[2:].strip())
netflix_df.head()

,Title,Date,Episode,titleId,seasonNumber
0,WWII in Color: Road to Victory,14/02/2022,Ocupação da Itália,tt16477402,1
1,WWII in Color: Road to Victory,08/02/2022,Batalha de Kursk,tt16477402,1
2,WWII in Color: Road to Victory,31/01/2022,Ocupação do Norte da África,tt16477402,1
3,WWII in Color: Road to Victory,30/12/2021,Batalha do Atlântico,tt16477402,1
4,WWII in Color: Road to Victory,27/12/2021,Dunquerque,tt16477402,1


## Imputar os episódios do título "Nova"

In [42]:
netflix_df.loc[132,'Episode'] = 'Death Dive to Saturn'
netflix_df.loc[132,'seasonNumber'] = 44
netflix_df.loc[133,'Episode'] = 'Black Hole Apocalypse'
netflix_df.loc[133,'seasonNumber'] = 45

In [43]:
netflix_df.loc[[132, 133]]

,Title,Date,Episode,titleId,seasonNumber
132,Nova,14/06/2020,Death Dive to Saturn,tt0206501,44
133,Nova,13/06/2020,Black Hole Apocalypse,tt0206501,45


## Adicionar coluna de episodeNumber

In [44]:
netflix_df.head()

,Title,Date,Episode,titleId,seasonNumber
0,WWII in Color: Road to Victory,14/02/2022,Ocupação da Itália,tt16477402,1
1,WWII in Color: Road to Victory,08/02/2022,Batalha de Kursk,tt16477402,1
2,WWII in Color: Road to Victory,31/01/2022,Ocupação do Norte da África,tt16477402,1
3,WWII in Color: Road to Victory,30/12/2021,Batalha do Atlântico,tt16477402,1
4,WWII in Color: Road to Victory,27/12/2021,Dunquerque,tt16477402,1


In [45]:
def list_range(start, finish, step):
  return list(range(start, finish-1, step))

In [46]:
ep_numbers = []

ep_numbers.extend(list_range(5, 1, -1))   # WWII in Color: Road to Victory
ep_numbers.extend(list_range(10, 5, -1))  # Greatest Events of WWII in Colour
ep_numbers.extend([1])                    # The Universe
ep_numbers.extend(list_range(4, 1, -1))   # Greatest Events of WWII in Colour
ep_numbers.extend(list_range(11, 1, -1))  # The Walking Dead
ep_numbers.extend(list_range(22, 19, -1)) # Sangatsu no Lion
ep_numbers.extend(list_range(4, 1, -1))   # Challenger: The Final Flight
ep_numbers.extend(list_range(18, 1, -1))  # Sangatsu no Lion
ep_numbers.extend(list_range(10, 1, -1))  # Space Force
ep_numbers.extend(list_range(22, 3, -1))  # Sangatsu no Lion
ep_numbers.extend([np.nan])               # Mission Control: The Unsung Heroes of Apollo
ep_numbers.extend([2, 1])                 # Sangatsu no Lion
ep_numbers.extend([1])                    # The Martian
ep_numbers.extend(list_range(25, 1, -1))  # Toradora!
ep_numbers.extend(list_range(9, 1, -1))  # History 101
ep_numbers.extend([np.nan])               # A Whisker Away
ep_numbers.extend([3])                    # Coronavirus, Explained
ep_numbers.extend([6])                    # The Martian
ep_numbers.extend([2])                    # Coronavirus, Explained
ep_numbers.extend(list_range(5, 2, -1))   # The Martian
ep_numbers.extend([np.nan])               # The Search for Life in Space
ep_numbers.extend([1])                    # Coronavirus, Explained
ep_numbers.extend([19, 1])                # Nova: Death Dive to Saturn (S44, E19) | Nova: Black Hole Apocalypse (S45, E1)
ep_numbers.extend(list_range(13, 1, -1))  # The Universe
ep_numbers.extend([np.nan, np.nan])       # Sen to Chihiro no kamikakushi e Tonari no Totoro
ep_numbers.extend(list_range(6, 1, -1))   # Full House (S5)
ep_numbers.extend([26])                   # Full House (S4)
ep_numbers.extend([1])                    # Full House (S1)
ep_numbers.extend(list_range(25, 12, -1)) # Full House (S4)
ep_numbers.extend([np.nan])               # Kimi no na wa.
ep_numbers.extend(list_range(11, 4, -1))  # Full House (S4)
ep_numbers.extend([np.nan])               # Flavors of Youth
ep_numbers.extend(list_range(3, 1, -1))   # Full House (S4)
ep_numbers.extend(list_range(24, 12, -1)) # Full House (S3)
ep_numbers.extend(list_range(12, 10, -1)) # Karakai Jouzu no Takagi-san (S2)
ep_numbers.extend([11])                   # Full House (S3)
ep_numbers.extend([9, 8])                 # Karakai Jouzu no Takagi-san (S2)
ep_numbers.extend([10])                   # Full House (S3)
ep_numbers.extend([7, 6])                 # Karakai Jouzu no Takagi-san (S2)
ep_numbers.extend([9])                    # Full House (S3)
ep_numbers.extend([5])                    # Karakai Jouzu no Takagi-san (S2)
ep_numbers.extend([8])                    # Full House (S3)
ep_numbers.extend([4])                    # Karakai Jouzu no Takagi-san (S2)
ep_numbers.extend([7])                    # Full House (S3)
ep_numbers.extend([3])                    # Karakai Jouzu no Takagi-san (S2)
ep_numbers.extend([6, 5])                 # Full House (S3)
ep_numbers.extend([2])                    # Karakai Jouzu no Takagi-san (S2)
ep_numbers.extend(list_range(4, 1, -1))   # Full House (S3)
ep_numbers.extend(list_range(22, 1, -1))  # Full House (S2)
ep_numbers.extend(list_range(22, 2, -1))  # Full House (S1)
ep_numbers.extend([1])                    # SpongeBob SquarePants (S6)
ep_numbers.extend([1])                    # Karakai Jouzu no Takagi-san (S2)
ep_numbers.extend([np.nan])               # School of Rock
ep_numbers.extend(list_range(3, 1, -1))   # Sherlock (S4)
ep_numbers.extend([0, 3, 2, 1])           # Sherlock (S3)
ep_numbers.extend(list_range(3, 1, -1))   # Sherlock (S2)
ep_numbers.extend(list_range(3, 1, -1))   # Sherlock (S1)
ep_numbers.extend([np.nan])               # Atlantis: The Lost Empire
ep_numbers.extend(list_range(24, 8, -1))  # The Fresh Prince of Bel-Air (S6)
ep_numbers.extend(list_range(25, 11, -1)) # The Fresh Prince of Bel-Air (S5) ?
ep_numbers.extend(list_range(24, 1, -1)) # The Fresh Prince of Bel-Air (S4) 26
ep_numbers.extend([24, 23, 20, 16, 13, 12, 11, 9, 7, 6, 5, 4, 2, 1]) # The Fresh Prince of Bel-Air (S3) ?
ep_numbers.extend(list_range(24, 7, -1))  # The Fresh Prince of Bel-Air (S2) ?
ep_numbers.extend(list_range(19, 1, -1))  # The Fresh Prince of Bel-Air (S1) ?
ep_numbers.extend([np.nan, np.nan])       # Little Witch Academia: The Enchanted Parade e Little Witch Academia
ep_numbers.extend(list_range(12, 1, -1))  # Little Witch Academia (S2)
ep_numbers.extend(list_range(13, 1, -1))  # Little Witch Academia (S1)
ep_numbers.extend([np.nan])               # Luzes no Céu: Fireworks
ep_numbers.extend(list_range(11, 1, -1))  # Anohana: The Flower We Saw That Day (S1)
ep_numbers.extend([np.nan, np.nan, np.nan, np.nan, np.nan]) # Tron: Legacy, Children Who Chase Lost Voices, Toki o kakeru shôjo, Wolf Children, Koe no katachi
ep_numbers.extend(list_range(12, 10, -1)) # One Punch Man: Wanpanman (S1)
ep_numbers.extend([np.nan])               # Inside Out
ep_numbers.extend(list_range(9, 1, -1))   # One Punch Man: Wanpanman (S1)
ep_numbers.extend([np.nan, np.nan, np.nan, np.nan, np.nan]) # Coraline, Kubo and the Two Strings, ParaNorman, The SpongeBob Movie: Sponge Out of Water, The Secret Life of Pets
ep_numbers.extend([1])                    # The Walking Dead (S1)
ep_numbers.extend([np.nan])               # Exterminadores do Além Contra a Loira do Banheiro

ep_numbers = np.array(ep_numbers, dtype='object')
print(len(ep_numbers))
print(ep_numbers)

449
[5 4 3 2 1 10 9 8 7 6 5 1 4 3 2 1 11 10 9 8 7 6 5 4 3 2 1 22 21 20 19 4 3
 2 1 18 17 16 15 14 13 12 11 10 9 8 7 6 5 4 3 2 1 10 9 8 7 6 5 4 3 2 1 22
 21 20 19 18 17 16 15 14 13 12 11 10 9 8 7 6 5 4 3 nan 2 1 1 25 24 23 22
 21 20 19 18 17 16 15 14 13 12 11 10 9 8 7 6 5 4 3 2 1 9 8 7 6 5 4 3 2 1
 nan 3 6 2 5 4 3 2 nan 1 19 1 13 12 11 10 9 8 7 6 5 4 3 2 1 nan nan 6 5 4
 3 2 1 26 1 25 24 23 22 21 20 19 18 17 16 15 14 13 12 nan 11 10 9 8 7 6 5
 4 nan 3 2 1 24 23 22 21 20 19 18 17 16 15 14 13 12 12 11 10 11 9 8 10 7 6
 9 5 8 4 7 3 6 5 2 4 3 2 1 22 21 20 19 18 17 16 15 14 13 12 11 10 9 8 7 6
 5 4 3 2 1 22 21 20 19 18 17 16 15 14 13 12 11 10 9 8 7 6 5 4 3 2 1 1 nan
 3 2 1 0 3 2 1 3 2 1 3 2 1 nan 24 23 22 21 20 19 18 17 16 15 14 13 12 11
 10 9 8 25 24 23 22 21 20 19 18 17 16 15 14 13 12 11 24 23 22 21 20 19 18
 17 16 15 14 13 12 11 10 9 8 7 6 5 4 3 2 1 24 23 20 16 13 12 11 9 7 6 5 4
 2 1 24 23 22 21 20 19 18 17 16 15 14 13 12 11 10 9 8 7 19 18 17 16 15 14
 13 12 11 10 9 8 7 6 5 4 3 2 1 nan n

In [47]:
netflix_df['episodeNumber'] = ep_numbers
netflix_df.head()

,Title,Date,Episode,titleId,seasonNumber,episodeNumber
0,WWII in Color: Road to Victory,14/02/2022,Ocupação da Itália,tt16477402,1,5
1,WWII in Color: Road to Victory,08/02/2022,Batalha de Kursk,tt16477402,1,4
2,WWII in Color: Road to Victory,31/01/2022,Ocupação do Norte da África,tt16477402,1,3
3,WWII in Color: Road to Victory,30/12/2021,Batalha do Atlântico,tt16477402,1,2
4,WWII in Color: Road to Victory,27/12/2021,Dunquerque,tt16477402,1,1


In [48]:
netflix_df.dtypes

Title            object
Date             object
Episode          object
titleId          object
seasonNumber     object
episodeNumber    object
dtype: object

## Trocar titleId de "Sherlock"

In [49]:
# Sherlock true ID: tt1475582
netflix_df['titleId'].mask(netflix_df['titleId'] == 'tt0337593', 'tt1475582', inplace=True)

## Trocar titleId do "The Walking Dead"

In [50]:
# The Walking Dead true ID: tt1520211
netflix_df['titleId'].mask(netflix_df['titleId'] == 'tt0028478', 'tt1520211', inplace=True)

## Trocar titleId do "Space Force"

In [51]:
# Space Force true ID: tt9612516
netflix_df['titleId'].mask(netflix_df['titleId'] == 'tt11309924', 'tt9612516', inplace=True)

## Trocar titleId do "The Martian"

In [52]:
# The Martian true ID: tt4939064
netflix_df['titleId'].mask(netflix_df['titleId'] == 'tt2910906', 'tt4939064', inplace=True)

## Preencher seasonNumber de "Challenger: The Final Flight"

In [62]:
netflix_df.loc[31:34, 'seasonNumber'] = 1
netflix_df.loc[31:34]

,Title,Date,Episode,titleId,seasonNumber,episodeNumber
31,Challenger: The Final Flight,15/02/2021,Nada acaba aqui,tt12930534,1,4
32,Challenger: The Final Flight,15/02/2021,Uma grande avaria,tt12930534,1,3
33,Challenger: The Final Flight,08/02/2021,Socorro!,tt12930534,1,2
34,Challenger: The Final Flight,07/02/2021,O espaço para todos,tt12930534,1,1


## Preencher seasonNumber de "Coronavirus, Explained"

In [68]:
netflix_df.loc[[123, 125, 131], 'seasonNumber'] = 1
netflix_df.loc[[123, 125, 131]]

,Title,Date,Episode,titleId,seasonNumber,episodeNumber
123,"Coronavirus, Explained",20/06/2020,Como encarar,tt12189310,1,3
125,"Coronavirus, Explained",18/06/2020,Corrida por uma vacina,tt12189310,1,2
131,"Coronavirus, Explained",14/06/2020,A atual pandemia,tt12189310,1,1


## Associar um episodeId

In [69]:
def get_episodeId_from_imdb(titleId, seasonNumber, episodeNumber):
  episodes = spark.sql(f"SELECT tconst FROM episode_df WHERE parentTconst = '{titleId}' AND seasonNumber = '{seasonNumber}' AND episodeNumber = '{episodeNumber}'").toPandas()
  episodes = episodes.drop_duplicates(subset=['tconst'])
  episodes = episodes.reset_index(drop=True)

  if len(episodes) > 0:
    id = episodes['tconst'][episodes.index[0]]
  else:
    id = None

  episodeId = id if id != None else np.nan
  return episodeId

In [70]:
episodeIds = netflix_df.apply(lambda x: get_episodeId_from_imdb(x['titleId'], x['seasonNumber'], x['episodeNumber']), axis=1)
episodeIds = episodeIds.tolist()
episodeIds

['tt16542940',
 'tt16542830',
 'tt16542592',
 'tt16542414',
 'tt16542412',
 'tt9103988',
 'tt9103970',
 'tt9103966',
 'tt9103964',
 'tt9103960',
 'tt9103958',
 'tt1119104',
 'tt9103954',
 'tt9103952',
 'tt9103948',
 'tt9103946',
 'tt6337096',
 'tt6337088',
 'tt6337078',
 'tt6327118',
 'tt6325296',
 'tt6207930',
 'tt6198668',
 'tt6198662',
 'tt6198656',
 'tt6198638',
 'tt6156390',
 'tt8792878',
 'tt8792876',
 'tt8792870',
 'tt8792864',
 'tt12986336',
 'tt12986334',
 'tt12986328',
 'tt12937576',
 'tt8792858',
 'tt8792850',
 'tt8792844',
 'tt8792836',
 'tt8792832',
 'tt8792824',
 'tt8792818',
 'tt8792808',
 'tt8792582',
 'tt8792574',
 'tt8792568',
 'tt8792564',
 'tt8792554',
 'tt8792428',
 'tt8792422',
 'tt8792414',
 'tt8792382',
 'tt7282032',
 'tt10572536',
 'tt10572532',
 'tt10572530',
 'tt10572524',
 'tt10572522',
 'tt10572520',
 'tt10572516',
 'tt10572514',
 'tt10572508',
 'tt10572504',
 'tt7726116',
 'tt7726114',
 'tt7726110',
 'tt7726106',
 'tt7726100',
 'tt7726098',
 'tt7726096',
 

In [71]:
netflix_df['episodeId'] = episodeIds
netflix_df.head()

,Title,Date,Episode,titleId,seasonNumber,episodeNumber,episodeId
0,WWII in Color: Road to Victory,14/02/2022,Ocupação da Itália,tt16477402,1,5,tt16542940
1,WWII in Color: Road to Victory,08/02/2022,Batalha de Kursk,tt16477402,1,4,tt16542830
2,WWII in Color: Road to Victory,31/01/2022,Ocupação do Norte da África,tt16477402,1,3,tt16542592
3,WWII in Color: Road to Victory,30/12/2021,Batalha do Atlântico,tt16477402,1,2,tt16542414
4,WWII in Color: Road to Victory,27/12/2021,Dunquerque,tt16477402,1,1,tt16542412


## Organizar colunas

In [72]:
netflix_df = netflix_df[['Date', 'titleId', 'Title', 'episodeId', 'Episode', 'seasonNumber', 'episodeNumber']]
netflix_df.head()

,Date,titleId,Title,episodeId,Episode,seasonNumber,episodeNumber
0,14/02/2022,tt16477402,WWII in Color: Road to Victory,tt16542940,Ocupação da Itália,1,5
1,08/02/2022,tt16477402,WWII in Color: Road to Victory,tt16542830,Batalha de Kursk,1,4
2,31/01/2022,tt16477402,WWII in Color: Road to Victory,tt16542592,Ocupação do Norte da África,1,3
3,30/12/2021,tt16477402,WWII in Color: Road to Victory,tt16542414,Batalha do Atlântico,1,2
4,27/12/2021,tt16477402,WWII in Color: Road to Victory,tt16542412,Dunquerque,1,1


## Exportar dados

In [73]:
netflix_df.to_csv('NetflixIMDb.csv', index=False, encoding='utf-8') 
files.download('NetflixIMDb.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Exportar dados do IMDb

In [ ]:
imdb_ratings = spark.sql(f"SELECT * FROM ratings_df WHERE tconst IN {tuple(title_ids)}").toPandas()
imdb_ratings.shape

(41, 3)

In [ ]:
imdb_basics = spark.sql(f"SELECT * FROM basics_df WHERE tconst IN {tuple(title_ids)}").toPandas()
imdb_basics.shape

(43, 9)

In [ ]:
imdb_episode = spark.sql(f"SELECT * FROM episode_df WHERE parentTconst IN {tuple(title_ids)}").toPandas()
imdb_episode.shape

(1879, 4)

## Download

In [ ]:
imdb_ratings.to_csv('IMDbRatings.csv', index=False, encoding='utf-8')
imdb_basics.to_csv('IMDbBasics.csv', index=False, encoding='utf-8')
imdb_episode.to_csv('IMDbEpisode.csv', index=False, encoding='utf-8')

files.download('IMDbRatings.csv')
files.download('IMDbBasics.csv')
files.download('IMDbEpisode.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Separar gêneros em IMDbBasics

In [ ]:
# IMDbBasics DataFrame
imdb_basics_df = pd.read_csv('https://raw.githubusercontent.com/vichShir/netflix-analysis/master/data/IMDb/IMDbBasics.csv')

In [ ]:
imdb_basics_df = imdb_basics_df.drop('genres', axis=1)\
                          .join(imdb_basics_df['genres']
                          .str.split(',', expand=True)
                          .stack().reset_index(level=1, drop=True)
                          .rename('genres'))
imdb_basics_df.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0028478,movie,The Walking Dead,The Walking Dead,0,1936,\N,66,Crime
0,tt0028478,movie,The Walking Dead,The Walking Dead,0,1936,\N,66,Drama
0,tt0028478,movie,The Walking Dead,The Walking Dead,0,1936,\N,66,Horror
1,tt0073174,movie,Inside Out,Inside Out,0,1975,\N,97,Comedy
1,tt0073174,movie,Inside Out,Inside Out,0,1975,\N,97,Crime


In [ ]:
imdb_basics_df.to_csv('IMDbBasics.csv', index=False, encoding='utf-8')
files.download('IMDbBasics.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Finalizar sessão do Spark

In [ ]:
spark.stop()

# Área de Testes

In [ ]:
# Registering a table
akas_df.registerTempTable("akas_df")
spark.sql("SELECT * FROM akas_df").show(3)

/usr/local/lib/python3.7/dist-packages/pyspark/sql/dataframe.py:140: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  FutureWarning


+---------+--------+--------------------+------+--------+-----------+-------------+---------------+
|  titleId|ordering|               title|region|language|      types|   attributes|isOriginalTitle|
+---------+--------+--------------------+------+--------+-----------+-------------+---------------+
|tt0000001|       1|          Карменсіта|    UA|      \N|imdbDisplay|           \N|              0|
|tt0000001|       2|          Carmencita|    DE|      \N|         \N|literal title|              0|
|tt0000001|       3|Carmencita - span...|    HU|      \N|imdbDisplay|           \N|              0|
+---------+--------+--------------------+------+--------+-----------+-------------+---------------+
only showing top 3 rows



In [32]:
spark.sql("SELECT * FROM akas_df WHERE title = 'Mars' AND isOriginalTitle = '1'").toPandas()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0137094,1,Mars,\N,\N,original,\N,1
1,tt0142647,1,Mars,\N,\N,original,\N,1
2,tt0342739,1,Mars,\N,\N,original,\N,1
3,tt0417949,1,Mars,\N,\N,original,\N,1
4,tt0855874,1,Mars,\N,\N,original,\N,1
5,tt1109488,1,Mars,\N,\N,original,\N,1
6,tt1204100,1,Mars,\N,\N,original,\N,1
7,tt13561190,1,Mars,\N,\N,original,\N,1
8,tt1692873,1,Mars,\N,\N,original,\N,1
9,tt2121319,1,Mars,\N,\N,original,\N,1


In [ ]:
ratings_df.registerTempTable("ratings_df")
spark.sql("SELECT * FROM ratings_df WHERE tconst = 'tt1475582'").show()

/usr/local/lib/python3.7/dist-packages/pyspark/sql/dataframe.py:140: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  FutureWarning


+---------+-------------+--------+
|   tconst|averageRating|numVotes|
+---------+-------------+--------+
|tt1475582|          9.1|  882560|
+---------+-------------+--------+



In [ ]:
basics_df.registerTempTable("basics_df")
spark.sql("SELECT * FROM basics_df WHERE tconst = 'tt0582989'").toPandas()

/usr/local/lib/python3.7/dist-packages/pyspark/sql/dataframe.py:140: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  FutureWarning


,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0582989,tvEpisode,"I, Done: Part 1","I, Done: Part 1",0,1996,\N,23,Comedy


In [63]:
episode_df.registerTempTable("episode_df")
spark.sql("SELECT * FROM episode_df WHERE parentTconst = 'tt12189310'").toPandas()

/usr/local/lib/python3.7/dist-packages/pyspark/sql/dataframe.py:140: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  FutureWarning


,tconst,parentTconst,seasonNumber,episodeNumber
0,tt12214632,tt12189310,1,1
1,tt12228072,tt12189310,1,2
2,tt12228074,tt12189310,1,3
